In [48]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "sample.osm"
postcode_NV_re = re.compile(r'[A-Z]+')
postcode_re = re.compile(r'-')


def audit_postcode_type(pt_types, pt_name):
    m = re.search(postcode_NV_re, pt_name)
    n = re.search(postcode_re, pt_name)
    
    if m:
        pt_type = m.group()
        pt_types[pt_type].add(pt_name)
    elif n:
        pt_type = n.group()
        pt_types[pt_type].add(pt_name)

def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    pt_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        for tag in elem.iter("tag"):
            if is_postcode(tag):
                audit_postcode_type(pt_types, tag.attrib['v'])
    osm_file.close()
    return pt_types


def update_name(name):
    m = re.search(postcode_NV_re, name)
    n = re.search(postcode_re, name)
    if m:
        name = (re.sub(postcode_NV_re, '', name)).strip()
    elif n:
        name = name.split('-')[0]
    return name


def test():
    postcode_types = audit(OSMFILE)
#    assert len(postcode_types) == 2
    pprint.pprint(dict(postcode_types))

    for postcode_type, postcode in postcode_types.iteritems():
        for name in postcode:
            better_name = update_name(name)
            print name, "=>", better_name
            if name == "89108-7049":
                assert better_name == "89108"
            if name == "NV 89149":
                assert better_name == "89149"


if __name__ == '__main__':
    test()

{'-': set(['89108-7049']), 'NV': set(['NV 89149'])}
89108-7049 => 89108
NV 89149 => 89149
